# Task 2: ETL Process Implementation


This notebook implements an **Extract–Transform–Load (ETL)** pipeline for building a retail data warehouse from raw transactional data.

The source data used in this task is the **Online Retail dataset** obtained from the **UCI Machine Learning Repository**. The dataset contains detailed transaction records for an online retail business, including invoice information, product descriptions, quantities, prices, customer identifiers, countries, and transaction dates.

The purpose of this ETL process is to:
- Extract raw retail transaction data from a CSV file
- Clean and transform the data to improve quality and consistency
- Enrich the data with derived attributes to support deeper analysis
- Structure the data into dimension and fact tables following a **star schema**
- Load the processed data into a **SQLite-based data warehouse** for analytical querying

As part of the transformation phase, an additional **product category attribute** is created to enhance analytical capabilities. Since the original dataset does not provide predefined product categories, **text mining techniques** are applied to product descriptions using **TF-IDF vectorization and K-Means clustering**. This enables automated grouping of products into meaningful categories, supporting category-level sales analysis.

By the end of the ETL process, the resulting data warehouse is optimized for business intelligence and data mining tasks such as customer analysis, product performance evaluation, and time-based sales reporting.


In [2]:
# Importing libraries
import pandas as pd
import sqlite3
from datetime import datetime
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()


## 1. Data Extraction

### Extract Data from CSV




The Extract phase involves loading raw transactional data from an external CSV file into the ETL environment.

During this phase:
- The CSV file is read into a pandas DataFrame
- Missing values in key attributes are handled
- Data types such as dates and numeric fields are standardized

The objective of this phase is to accurately capture source data in a structured format suitable for transformation.


In [ ]:
# creating a function to extract data from the csv file
def extract_data(file_path):
    logging.info("Starting data extraction...")
    try:
        df = pd.read_csv(file_path, encoding='ISO-8859-1')
        logging.info(f"Successfully loaded {len(df)} rows from CSV")
        
        logging.info("Handling missing values...")
        df['Description'] = df['Description'].fillna('')
        df['CustomerID'] = df['CustomerID'].fillna(0).astype(int)
        
        logging.info("Converting data types...")
        df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
        df['UnitPrice'] = pd.to_numeric(df['UnitPrice'], errors='coerce')
        df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
        
        logging.info("Extraction completed successfully")
        return df
    except Exception as e:
        logging.error(f"Error during extraction: {e}")
        raise
# function is call
csv_file = 'online_retail.csv'
df = extract_data(csv_file)

print(f"\nDataset shape: {df.shape}")
print("\nFirst few rows:")
df.head()

2025-12-14 17:29:46,835 - INFO - Starting data extraction...
2025-12-14 17:29:47,286 - INFO - Successfully loaded 541909 rows from CSV
2025-12-14 17:29:47,286 - INFO - Handling missing values...
2025-12-14 17:29:47,319 - INFO - Converting data types...
2025-12-14 17:29:47,413 - INFO - Extraction completed successfully



Dataset shape: (541909, 8)

First few rows:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


### Check Data Quality

In [5]:
print("Dataset Information:")
print(df.info())

print("Missing Values:")
print(df.isnull().sum())

print("Basic Statistics:")
print(df.describe())

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  541909 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   541909 non-null  int32         
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), int64(1), object(4)
memory usage: 31.0+ MB
None
Missing Values:
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64
Basic Statistics:
            Quantity                    InvoiceDate      UnitPrice  \
count  541909.00000

## 2. Data Transformation


The Transform phase cleans, enriches, and restructures the extracted data to meet analytical and data warehousing requirements.

Key transformation activities include:
- Data quality assessment and removal of invalid records
- Feature engineering such as calculating total sales values
- Product categorization using text mining techniques (TF-IDF and K-Means)
- Temporal filtering of records
- Construction of dimension tables (Customer, Product, Time)
- Preparation of the Sales Fact table with derived measures

This phase converts raw transactional data into a structured star schema format suitable for analysis.


In [ ]:
# Remove Outliers and Invalid Records
logging.info("Removing outliers and invalid records...")
initial_count = len(df)
df_clean = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)].copy()
removed_count = initial_count - len(df_clean)

logging.info(f"Removed {removed_count} invalid records")
print(f"Original records: {initial_count}")
print(f"Valid records: {len(df_clean)}")
print(f"Removed records: {removed_count}")

2025-12-14 18:13:44,827 - INFO - Removing outliers and invalid records...
2025-12-14 18:13:44,906 - INFO - Removed 11805 invalid records


Original records: 541909
Valid records: 530104
Removed records: 11805


### Calculate TotalSales

In [7]:
logging.info("Calculating TotalSales...")
df_clean['TotalSales'] = df_clean['Quantity'] * df_clean['UnitPrice']

print("TotalSales calculated successfully")
print("\nSample of data with TotalSales:")
df_clean[['InvoiceNo', 'Description', 'Quantity', 'UnitPrice', 'TotalSales']].head(10)

2025-12-14 18:15:00,369 - INFO - Calculating TotalSales...


TotalSales calculated successfully

Sample of data with TotalSales:


,InvoiceNo,Description,Quantity,UnitPrice,TotalSales
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,15.30
1,536365,WHITE METAL LANTERN,6,3.39,20.34
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2.75,22.00
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,20.34
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,20.34
5,536365,SET 7 BABUSHKA NESTING BOXES,2,7.65,15.30
6,536365,GLASS STAR FROSTED T-LIGHT HOLDER,6,4.25,25.50
7,536366,HAND WARMER UNION JACK,6,1.85,11.10
8,536366,HAND WARMER RED POLKA DOT,6,1.85,11.10
9,536367,ASSORTED COLOUR BIRD ORNAMENT,32,1.69,54.08


### Add Product Categories Using TF-IDF and KMeans

In [8]:
logging.info("Adding product categories using TF-IDF and KMeans clustering...")

tfidf = TfidfVectorizer(stop_words='english', max_features=2000)
tfidf_matrix = tfidf.fit_transform(df_clean['Description'])

k = 8
kmeans = KMeans(n_clusters=k, random_state=42)
df_clean['ProductCategory'] = kmeans.fit_predict(tfidf_matrix)

logging.info("Product categorization completed")
print(f"Created {k} product categories")
print("\nProduct category distribution:")
print(df_clean['ProductCategory'].value_counts().sort_index())

2025-12-14 18:15:39,642 - INFO - Adding product categories using TF-IDF and KMeans clustering...
2025-12-14 18:15:44,662 - INFO - Product categorization completed


Created 8 product categories

Product category distribution:
ProductCategory
0     44226
1     22697
2     15383
3    315565
4     40918
5     66390
6     18532
7      6393
Name: count, dtype: int64


### Examine Sample Products in Each Category

In [9]:
unique_categories = df_clean['ProductCategory'].unique()

print("Sample product descriptions for each category:\n")
for category in sorted(unique_categories):
    print(f"Category {category} examples:")
    samples = df_clean[df_clean['ProductCategory'] == category]['Description'].head(5)
    print(samples.to_string(index=False))
    print("-" * 60)

Sample product descriptions for each category:

Category 0 examples:
WHITE HANGING HEART T-LIGHT HOLDER
    RED WOOLLY HOTTIE WHITE HEART.
 GLASS STAR FROSTED T-LIGHT HOLDER
       RECIPE BOX WITH METAL HEART
WHITE HANGING HEART T-LIGHT HOLDER
------------------------------------------------------------
Category 1 examples:
            SPACEBOY LUNCH BOX 
        LUNCH BOX I LOVE LONDON
       CIRCUS PARADE LUNCH BOX 
CHARLOTTE BAG DOLLY GIRL DESIGN
       STRAWBERRY CHARLOTTE BAG
------------------------------------------------------------
Category 2 examples:
          JUMBO BAG PINK POLKADOT
BLUE 3 PIECE POLKADOT CUTLERY SET
           TOY TIDY PINK POLKADOT
     BLACK/BLUE POLKADOT UMBRELLA
               POLKADOT RAIN HAT 
------------------------------------------------------------
Category 3 examples:
     CREAM CUPID HEARTS COAT HANGER
KNITTED UNION FLAG HOT WATER BOTTLE
             HAND WARMER UNION JACK
          HAND WARMER RED POLKA DOT
      ASSORTED COLOUR BIRD ORNAMENT


### Map Category Numbers to Meaningful Names

In [10]:
category_mapping = {
    1: "Candles & Lighting Decor",
    4: "Home Metal Decor",
    3: "Warm Accessories & Gift Items",
    5: "Gift Sets & DIY Craft Kits",
    6: "Mugs & Home Messages",
    7: "Fashion & Personal Accessories",
    0: "Party & Celebration Supplies",
    2: "Bags & Storage Accessories"
}

df_clean['ProductCategoryName'] = df_clean['ProductCategory'].map(category_mapping)

print("Category mapping applied successfully")
print("\nProduct category name distribution:")
print(df_clean['ProductCategoryName'].value_counts())
print("\nSample with categories:")
df_clean[['Description', 'ProductCategory', 'ProductCategoryName']].head(10)

Category mapping applied successfully

Product category name distribution:
ProductCategoryName
Warm Accessories & Gift Items     315565
Gift Sets & DIY Craft Kits         66390
Party & Celebration Supplies       44226
Home Metal Decor                   40918
Candles & Lighting Decor           22697
Mugs & Home Messages               18532
Bags & Storage Accessories         15383
Fashion & Personal Accessories      6393
Name: count, dtype: int64

Sample with categories:


,Description,ProductCategory,ProductCategoryName
0,WHITE HANGING HEART T-LIGHT HOLDER,0,Party & Celebration Supplies
1,WHITE METAL LANTERN,6,Mugs & Home Messages
2,CREAM CUPID HEARTS COAT HANGER,3,Warm Accessories & Gift Items
3,KNITTED UNION FLAG HOT WATER BOTTLE,3,Warm Accessories & Gift Items
4,RED WOOLLY HOTTIE WHITE HEART.,0,Party & Celebration Supplies
5,SET 7 BABUSHKA NESTING BOXES,5,Gift Sets & DIY Craft Kits
6,GLASS STAR FROSTED T-LIGHT HOLDER,0,Party & Celebration Supplies
7,HAND WARMER UNION JACK,3,Warm Accessories & Gift Items
8,HAND WARMER RED POLKA DOT,3,Warm Accessories & Gift Items
9,ASSORTED COLOUR BIRD ORNAMENT,3,Warm Accessories & Gift Items


### Filter Data for Last Year

In [19]:
max_date = df_clean['InvoiceDate'].max()
one_year_ago = max_date - pd.DateOffset(years=1)

df_filtered = df_clean[df_clean['InvoiceDate'] >= one_year_ago].copy()


logging.info(f"Filtered to {len(df_filtered)} records from last year")
print(f"Date range in original data: {df_clean['InvoiceDate'].min()} to {df_clean['InvoiceDate'].max()}")
print(f"Date range in filtered data: {df_filtered['InvoiceDate'].min()} to {df_filtered['InvoiceDate'].max()}")
print(f"Total filtered records: {len(df_filtered)}")

2025-12-14 19:06:13,856 - INFO - Filtered to 488624 records from last year


Date range in original data: 2011-01-04 10:00:00 to 2011-12-09 12:50:00
Date range in filtered data: 2011-01-04 10:00:00 to 2011-12-09 12:50:00
Total filtered records: 488624


### Create Customer Dimension

In [25]:
logging.info("Creating customer dimension data")

customer_dim = df_clean.groupby('CustomerID').agg({
    'Country': 'first',
    'TotalSales': 'sum',
    'InvoiceNo': 'count'
}).reset_index()

customer_dim.columns = ['CustomerID', 'Country', 'TotalPurchases', 'OrderCount']
customer_dim = customer_dim[customer_dim['CustomerID'] > 0]

print(f"Customer dimension created with {len(customer_dim)} unique customers")
print("\nTop 10 customers by total purchases:")
customer_dim.nlargest(10, 'TotalPurchases')

2025-12-14 19:29:16,588 - INFO - Creating customer dimension data


Customer dimension created with 4219 unique customers

Top 10 customers by total purchases:


,CustomerID,Country,TotalPurchases,OrderCount
1654,14646,Netherlands,271614.14,2006
4087,18102,United Kingdom,231822.69,413
3637,17450,United Kingdom,192521.95,334
2937,16446,United Kingdom,168472.50,3
1841,14911,EIRE,136087.12,5384
56,12415,Australia,124914.53,714
1305,14156,EIRE,117057.43,1397
3678,17511,United Kingdom,84351.30,875
1,12346,United Kingdom,77183.60,1
2637,16029,United Kingdom,67912.32,210


### Create Time Dimension

In [24]:
logging.info("Creating time dimension data")

df_filtered['Date'] = df_filtered['InvoiceDate'].dt.date
df_filtered['Day'] = df_filtered['InvoiceDate'].dt.day
df_filtered['Month'] = df_filtered['InvoiceDate'].dt.month
df_filtered['Quarter'] = df_filtered['InvoiceDate'].dt.quarter
df_filtered['Year'] = df_filtered['InvoiceDate'].dt.year
df_filtered['DayOfWeek'] = df_filtered['InvoiceDate'].dt.day_name()

time_dim = df_filtered[['Date', 'Day', 'Month', 'Quarter', 'Year', 'DayOfWeek']].drop_duplicates()
time_dim = time_dim.reset_index(drop=True)
time_dim['TimeID'] = time_dim.index + 1
time_dim = time_dim[['TimeID', 'Date', 'Day', 'Month', 'Quarter', 'Year', 'DayOfWeek']]

print(f"Time dimension created with {len(time_dim)} unique dates")
print("\nQuarter distribution:")
print(time_dim.groupby(['Year', 'Quarter']).size())
print("\nSample time data:")
time_dim.head(10)

2025-12-14 19:29:08,445 - INFO - Creating time dimension data


Time dimension created with 285 unique dates

Quarter distribution:
Year  Quarter
2011  1          75
      2          72
      3          78
      4          60
dtype: int64

Sample time data:


,TimeID,Date,Day,Month,Quarter,Year,DayOfWeek
0,1,2011-01-04,4,1,1,2011,Tuesday
1,2,2011-01-05,5,1,1,2011,Wednesday
2,3,2011-01-06,6,1,1,2011,Thursday
3,4,2011-01-07,7,1,1,2011,Friday
4,5,2011-01-09,9,1,1,2011,Sunday
5,6,2011-01-10,10,1,1,2011,Monday
6,7,2011-01-11,11,1,1,2011,Tuesday
7,8,2011-01-12,12,1,1,2011,Wednesday
8,9,2011-01-13,13,1,1,2011,Thursday
9,10,2011-01-14,14,1,1,2011,Friday


### Create Product Dimension

In [22]:
logging.info("Creating product dimension data")

product_dim = df_clean[['StockCode', 'Description', 'ProductCategoryName', 'UnitPrice']].drop_duplicates(subset=['StockCode'])
product_dim = product_dim[product_dim['StockCode'].notna()]
product_dim = product_dim.reset_index(drop=True)
product_dim['ProductID'] = product_dim.index + 1
product_dim.columns = ['StockCode', 'ProductName', 'ProductCategory', 'UnitPrice', 'ProductID']
product_dim = product_dim[['ProductID', 'StockCode', 'ProductName', 'ProductCategory', 'UnitPrice']]

print(f"Product dimension created with {len(product_dim)} unique products")
print("\nProducts per category:")
print(product_dim['ProductCategory'].value_counts())
print("\nSample product data:")
product_dim.head(10)

2025-12-14 19:06:47,990 - INFO - Creating product dimension data


Product dimension created with 3828 unique products

Products per category:
ProductCategory
Warm Accessories & Gift Items     2929
Gift Sets & DIY Craft Kits         338
Party & Celebration Supplies       257
Home Metal Decor                    95
Bags & Storage Accessories          78
Mugs & Home Messages                73
Candles & Lighting Decor            37
Fashion & Personal Accessories      21
Name: count, dtype: int64

Sample product data:


,ProductID,StockCode,ProductName,ProductCategory,UnitPrice
0,1,22386,JUMBO BAG PINK POLKADOT,Bags & Storage Accessories,1.95
1,2,21499,BLUE POLKADOT WRAP,Bags & Storage Accessories,0.42
2,3,21498,RED RETROSPOT WRAP,Home Metal Decor,0.42
3,4,22379,RECYCLING BAG RETROSPOT,Home Metal Decor,2.10
4,5,20718,RED RETROSPOT SHOPPER BAG,Home Metal Decor,1.25
5,6,85099B,JUMBO BAG RED RETROSPOT,Home Metal Decor,1.95
6,7,20682,RED RETROSPOT CHILDRENS UMBRELLA,Home Metal Decor,3.25
7,8,22961,JAM MAKING SET PRINTED,Gift Sets & DIY Craft Kits,1.45
8,9,22667,RECIPE BOX RETROSPOT,Home Metal Decor,2.95
9,10,22898,CHILDRENS APRON APPLES DESIGN,Warm Accessories & Gift Items,1.95


### Create Sales Fact Table

In [23]:
logging.info("Preparing sales fact data")

df_filtered = df_filtered.merge(
    time_dim[['Date', 'TimeID']], 
    on='Date', 
    how='left'
)

df_filtered = df_filtered.merge(
    product_dim[['StockCode', 'ProductID']], 
    on='StockCode', 
    how='left'
)

sales_fact = df_filtered[[
    'CustomerID', 'ProductID', 'TimeID', 
    'Quantity', 'TotalSales'
]].copy()

sales_fact = sales_fact.dropna()
sales_fact['SalesID'] = range(1, len(sales_fact) + 1)
sales_fact['Discount'] = 0.0
sales_fact['NetSales'] = sales_fact['TotalSales']

sales_fact = sales_fact[[
    'SalesID', 'CustomerID', 'ProductID', 'TimeID',
    'Quantity', 'TotalSales', 'Discount', 'NetSales'
]]

sales_fact['CustomerID'] = sales_fact['CustomerID'].astype(int)
sales_fact['ProductID'] = sales_fact['ProductID'].astype(int)
sales_fact['TimeID'] = sales_fact['TimeID'].astype(int)
sales_fact['Quantity'] = sales_fact['Quantity'].astype(int)
sales_fact['SalesID'] = sales_fact['SalesID'].astype(int)

print(f"Sales fact table created with {len(sales_fact)} records")
print("\nSales summary statistics:")
print(sales_fact[['Quantity', 'TotalSales', 'NetSales']].describe())
print("\nSample sales fact data:")
sales_fact.head(10)

2025-12-14 19:28:56,769 - INFO - Preparing sales fact data


Sales fact table created with 488624 records

Sales summary statistics:
            Quantity     TotalSales       NetSales
count  488624.000000  488624.000000  488624.000000
mean       10.701760      20.144198      20.144198
std       161.630065     280.166359     280.166359
min         1.000000       0.001000       0.001000
25%         1.000000       3.900000       3.900000
50%         4.000000       9.900000       9.900000
75%        12.000000      17.700000      17.700000
max     80995.000000  168469.600000  168469.600000

Sample sales fact data:


,SalesID,CustomerID,ProductID,TimeID,Quantity,TotalSales,Discount,NetSales
0,1,13313,1,1,10,19.5,0.0,19.5
1,2,13313,2,1,25,10.5,0.0,10.5
2,3,13313,3,1,25,10.5,0.0,10.5
3,4,13313,4,1,5,10.5,0.0,10.5
4,5,13313,5,1,10,12.5,0.0,12.5
5,6,13313,6,1,10,19.5,0.0,19.5
6,7,13313,7,1,6,19.5,0.0,19.5
7,8,13313,8,1,12,17.4,0.0,17.4
8,9,13313,9,1,6,17.7,0.0,17.7
9,10,13313,10,1,8,15.6,0.0,15.6


## 3. Data Loading

 ### Load Data to SQLite Database - Create Tables


The Load phase stores the transformed data into a SQLite data warehouse.

This phase includes:
- Creating dimension and fact tables according to the star schema design
- Loading dimension tables first to preserve referential integrity
- Populating the sales fact table with transactional measures
- Creating indexes to optimize query performance

At the end of this phase, the data warehouse is fully populated and ready for analytical querying.


In [26]:
logging.info("Starting data loading to database...")

db_path = 'retail_dw.db'

try:
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    logging.info("Dropping existing tables if any...")
    cursor.execute('DROP TABLE IF EXISTS SalesFact')
    cursor.execute('DROP TABLE IF EXISTS CustomerDim')
    cursor.execute('DROP TABLE IF EXISTS ProductDim')
    cursor.execute('DROP TABLE IF EXISTS TimeDim')
    
    logging.info("Creating dimension tables...")
    
    cursor.execute('''
        CREATE TABLE CustomerDim (
            CustomerID INTEGER PRIMARY KEY,
            Country TEXT,
            TotalPurchases REAL,
            OrderCount INTEGER
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE ProductDim (
            ProductID INTEGER PRIMARY KEY,
            StockCode TEXT,
            ProductName TEXT,
            ProductCategory TEXT,
            UnitPrice REAL
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE TimeDim (
            TimeID INTEGER PRIMARY KEY,
            Date TEXT,
            Day INTEGER,
            Month INTEGER,
            Quarter INTEGER,
            Year INTEGER,
            DayOfWeek TEXT
        )
    ''')
    
    logging.info("Creating fact table...")
    cursor.execute('''
        CREATE TABLE SalesFact (
            SalesID INTEGER PRIMARY KEY,
            CustomerID INTEGER,
            ProductID INTEGER,
            TimeID INTEGER,
            Quantity INTEGER,
            SalesAmount REAL,
            Discount REAL,
            NetSales REAL,
            FOREIGN KEY (CustomerID) REFERENCES CustomerDim(CustomerID),
            FOREIGN KEY (ProductID) REFERENCES ProductDim(ProductID),
            FOREIGN KEY (TimeID) REFERENCES TimeDim(TimeID)
        )
    ''')
    
    conn.commit()
    print("Tables created successfully")
    
except Exception as e:
    logging.error(f"Error creating tables: {e}")
    conn.close()
    raise

2025-12-14 19:34:40,161 - INFO - Starting data loading to database...
2025-12-14 19:34:40,163 - INFO - Dropping existing tables if any...
2025-12-14 19:34:40,166 - INFO - Creating dimension tables...
2025-12-14 19:34:40,209 - INFO - Creating fact table...


Tables created successfully


### Load Dimension Tables

In [27]:
logging.info("Loading customer dimension...")
customer_dim.to_sql('CustomerDim', conn, if_exists='append', index=False)
logging.info(f"Loaded {len(customer_dim)} customer records")

logging.info("Loading product dimension...")
product_dim.to_sql('ProductDim', conn, if_exists='append', index=False)
logging.info(f"Loaded {len(product_dim)} product records")

logging.info("Loading time dimension...")
time_dim.to_sql('TimeDim', conn, if_exists='append', index=False)
logging.info(f"Loaded {len(time_dim)} time records")

conn.commit()
print("All dimension tables loaded successfully")

2025-12-14 19:35:01,632 - INFO - Loading customer dimension...
2025-12-14 19:35:01,793 - INFO - Loaded 4219 customer records
2025-12-14 19:35:01,795 - INFO - Loading product dimension...
2025-12-14 19:35:01,805 - INFO - Loaded 3828 product records
2025-12-14 19:35:01,805 - INFO - Loading time dimension...
2025-12-14 19:35:01,814 - INFO - Loaded 285 time records


All dimension tables loaded successfully


In [29]:
sales_fact = sales_fact.rename(columns={
    'TotalSales': 'SalesAmount'
})


### Load Fact Table

In [30]:
logging.info("Loading sales fact")
sales_fact.to_sql('SalesFact', conn, if_exists='append', index=False)
conn.commit()
logging.info(f"Loaded {len(sales_fact)} sales records")

print("Sales fact table loaded successfully")

2025-12-14 20:09:53,738 - INFO - Loading sales fact
2025-12-14 20:09:54,434 - INFO - Loaded 488624 sales records


Sales fact table loaded successfully


### Create Indexes for Query Performance

In [31]:
logging.info("Creating indexes for better query performance...")

cursor.execute('CREATE INDEX IF NOT EXISTS idx_sales_customer ON SalesFact(CustomerID)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_sales_product ON SalesFact(ProductID)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_sales_time ON SalesFact(TimeID)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_product_category ON ProductDim(ProductCategory)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_customer_country ON CustomerDim(Country)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_time_quarter ON TimeDim(Quarter, Year)')

conn.commit()
logging.info("All indexes created successfully")

print("Indexes created for optimized query performance")

2025-12-14 20:10:35,124 - INFO - Creating indexes for better query performance...
2025-12-14 20:10:35,646 - INFO - All indexes created successfully


Indexes created for optimized query performance


### Verify Loaded Data


In [ ]:
logging.info("Verifying loaded data...")

cursor.execute("SELECT COUNT(*) FROM CustomerDim")
customer_count = cursor.fetchone()[0]
print(f"CustomerDim rows: {customer_count}")

cursor.execute("SELECT COUNT(*) FROM ProductDim")
product_count = cursor.fetchone()[0]
print(f"ProductDim rows: {product_count}")

cursor.execute("SELECT COUNT(*) FROM TimeDim")
time_count = cursor.fetchone()[0]
print(f"TimeDim rows: {time_count}")

cursor.execute("SELECT COUNT(*) FROM SalesFact")
sales_count = cursor.fetchone()[0]
print(f"SalesFact rows: {sales_count}")

print("\n" + "="*50)
print("Sample query - Top 5 customers by total purchases:")
sample_query = pd.read_sql_query('''
    SELECT CustomerID, Country, TotalPurchases, OrderCount
    FROM CustomerDim
    ORDER BY TotalPurchases DESC
    LIMIT 5
''', conn)
print(sample_query)

print("\n" + "="*50)
print("Sample query - Sales by product category:")
category_query = pd.read_sql_query('''
    SELECT p.ProductCategory, COUNT(*) as NumSales, SUM(s.NetSales) as TotalSales
    FROM SalesFact s
    JOIN ProductDim p ON s.ProductID = p.ProductID
    GROUP BY p.ProductCategory
    ORDER BY TotalSales DESC
'''mmjccccvvvdddxxxccccc
print(category_query)

2025-12-14 20:11:07,476 - INFO - Verifying loaded data...


CustomerDim rows: 4219
ProductDim rows: 3828
TimeDim rows: 285
SalesFact rows: 488624

Sample query - Top 5 customers by total purchases:
   CustomerID         Country  TotalPurchases  OrderCount
0       14646     Netherlands       271614.14        2006
1       18102  United Kingdom       231822.69         413
2       17450  United Kingdom       192521.95         334
3       16446  United Kingdom       168472.50           3
4       14911            EIRE       136087.12        5384

Sample query - Sales by product category:
                  ProductCategory  NumSales   TotalSales
0   Warm Accessories & Gift Items    288373  5847483.334
1      Gift Sets & DIY Craft Kits     62196  1125228.790
2                Home Metal Decor     38401   933374.880
3    Party & Celebration Supplies     40667   760863.660
4        Candles & Lighting Decor     21608   394652.670
5      Bags & Storage Accessories     14371   346997.230
6            Mugs & Home Messages     17145   289226.970
7  Fashion & Pe

### Close Database Connection

In [33]:
conn.close()
logging.info(f"Database connection closed. Database saved as {db_path}")

print(f"\nETL process completed successfully!")
print(f"Database file: {db_path}")
print("\nSummary:")
print(f"- Customers: {customer_count}")
print(f"- Products: {product_count}")
print(f"- Time periods: {time_count}")
print(f"- Sales transactions: {sales_count}")

2025-12-14 20:11:34,954 - INFO - Database connection closed. Database saved as retail_dw.db



ETL process completed successfully!
Database file: retail_dw.db

Summary:
- Customers: 4219
- Products: 3828
- Time periods: 285
- Sales transactions: 488624
